In [1]:
from pystackreg import StackReg
import numpy as np
import tifffile
from tifffile import memmap
import pandas as pd
import os, re
from PIL import Image
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

In [2]:
tiffs_directory = "/home/georgeos/Storage/Dropbox (Cambridge University)/DATA_Bakshi_PaulssonLab/PHASE_GC001/extracted"
if tiffs_directory[-1] != "/":
    tiffs_directory += "/"
tiff_names = os.listdir(tiffs_directory)
tiff_names.sort()
experiment_file_data = []
for name in tiff_names:
    file_data = re.split(' |\_| |\.| ',name)[:3]
    file_data[1] = file_data[1][1:]
    file_data.append(tiffs_directory + name)
    experiment_file_data.append(file_data)
experiment_file_data = pd.DataFrame(experiment_file_data)
experiment_file_data.columns = ["position", "time", "channel", "directory"]

In [3]:
experiment_file_data

,position,time,channel,directory
0,xy000,0000,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...
1,xy000,0001,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...
2,xy000,0002,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...
3,xy000,0003,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...
4,xy000,0004,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...
...,...,...,...,...
180456,xy112,1592,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...
180457,xy112,1593,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...
180458,xy112,1594,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...
180459,xy112,1595,BF,/home/georgeos/Storage/Dropbox (Cambridge Univ...


In [18]:
n_positions = len(experiment_file_data["position"].unique())
n_times = len(experiment_file_data["time"].unique())
n_channel = len(experiment_file_data["channel"].unique())
y_len, x_len = np.array(Image.open(experiment_file_data["directory"][0])).shape

In [19]:
positions = experiment_file_data["position"].unique()
times = experiment_file_data["time"].unique()
channels = experiment_file_data["channel"].unique()
reference_time = "0115"
reference_channel = "BF"

In [20]:
memmap_image = memmap(
    "/home/georgeos/Storage/Dropbox (Cambridge University)/DATA_Bakshi_PaulssonLab/PHASE_GC001/registered/registered.tif",
    shape=(n_positions, n_channel, n_times, y_len, x_len), 
    dtype=np.uint16,
    imagej=True)
def register_transform(memmap_image, time, reference_channel, reference_time):
    pos_val = 0
    for position in tqdm(positions):
        channel_val = 0
        reference_image_directory = experiment_file_data[(experiment_file_data["position"] == position) 
                 & (experiment_file_data["time"] == reference_time)
                 & (experiment_file_data["channel"] == reference_channel)
                ]["directory"].iloc[0]
        reference_image = np.array(Image.open(reference_image_directory))
        image_for_register_ref_channel = experiment_file_data[(experiment_file_data["position"] == position) 
                                                & (experiment_file_data["channel"] == reference_channel)
                                                & (experiment_file_data["time"] == time)
                                               ]
        ref_channel_image_dir = image_for_register_ref_channel["directory"].iloc[0]
        current_ref_channel_image = np.array(Image.open(ref_channel_image_dir))
        sr = StackReg(StackReg.RIGID_BODY)
        sr.register(reference_image, current_ref_channel_image)
        
        for channel in tqdm(channels):
            current_data = experiment_file_data[(experiment_file_data["position"] == position) 
                                                & (experiment_file_data["channel"] == channel)
                                                & (experiment_file_data["time"] == time)
                                               ]
            current_image_dir = current_data["directory"].iloc[0]
            current_image = np.array(Image.open(current_image_dir))
            out_tra = sr.transform(current_image)
            memmap_image[pos_val,channel_val,int(time),:,:] = out_tra
            channel_val += 1
        pos_val += 1

/home/georgeos/anaconda3/envs/SYMPTOMM/lib/python3.8/site-packages/tifffile/tifffile.py:2672: UserWarning: TiffWriter: truncating ImageJ file
  warnings.warn(


In [21]:
%%time
_ = Parallel(n_jobs=14)(delayed(register_transform)(
    memmap_image,
    time,reference_channel,
    reference_time
) for time in tqdm(times))

  0%|          | 0/1597 [00:00<?, ?it/s]

/home/georgeos/anaconda3/envs/SYMPTOMM/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 8min 44s, sys: 51.3 s, total: 9min 36s
Wall time: 9h 56min 25s
